In [1]:
import json
from tqdm import tqdm
import os
from random import choice
from itertools import groupby
import tensorflow as tf

/home/llluckygirl/envs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class KB(object):
    def __init__(self,kb_directory):
        print("start loading kb_data...")
        self.kb_directory = kb_directory
        self.id2kb,self.types,self.predicate = self.get_id2kb()
        self.kb2id = self.get_kb2id()
        self.kb = list(self.kb2id.keys())
        self.id = list(self.id2kb.keys())
        print("KB DATA INFORMATION")
        print("TOKEN SIZE:{}".format(self.get_token_size()))
        print("ID SIZE:{}".format(len(self)))
        print("TYPE SIZE:{}".format(len(self.types)))
        print("PREDICATE SIZE:{}".format(len(self.predicate)))
    def get_id2kb(self):
        print("construct id2kb dict...")
        id2kb = {}
        kbtype = set()
        predicate = set()
        multi_type = []
        with open(self.kb_directory) as f:
            for l in tqdm(f):
                tmp = json.loads(l)
                subject_id = tmp['subject_id']
                subject_alias = list(set([tmp['subject']] + tmp.get('alias', [])))
                subject_alias = [alias.lower() for alias in subject_alias]
                subject_type = [i.lower() for i in tmp['type']]
                kbtype.update(subject_type)
                try:
                    assert(len(tmp['type'])==1)
                except AssertionError:
                    multi_type.append(tmp['type'])
                subject_data = {}
                for i in tmp['data']:
                    predicate.add(i['predicate'].lower())
                    subject_data[i['predicate'].lower()] = i['object'].lower()
                if subject_data:
                    id2kb[subject_id] = {'alias': subject_alias, 'data': subject_data,'type':subject_type}
#         print(multi_type)
        return id2kb,kbtype,predicate
    def get_kb2id(self):
        print("construct kb2id dict...")
        kb2id = {}
        for i,j in self.id2kb.items():
            for k in j['alias']:
                if k not in kb2id:
                    kb2id[k] = []
                kb2id[k].append(i)
        return kb2id
    def __len__(self):
        return len(self.id2kb)
    def get_token_size(self):
        return len(self.kb)
#     def save(self):
        


In [3]:
kb_data = KB('./ccks2019_el/kb_data')

4355it [00:00, 21006.03it/s]

start loading kb_data...
construct id2kb dict...


399252it [00:18, 21052.67it/s]


construct kb2id dict...
KB DATA INFORMATION
TOKEN SIZE:303375
ID SIZE:399233
TYPE SIZE:51
PREDICATE SIZE:41841


In [4]:
print(len(kb_data))

399233


type2label = {'PER':['athlete','entertainmentperson','fictionalhuman',\
                     'person','human','familyname','organization'],
              'LOC':['nation','place','realestate','medicaldepartmenttype','country'],
              'ORG':['collegeoruniversity','scientificorganization','organism',\
                     'communicationmedium','internationalorganization','building','organization'],
              'MISC':['medicalcondition','animal','formula',\
                      'tvshow','event','academicdiscipline',\
                     'currency','tool','astronomicalobject',\
                     'brand','tvplay','vocabulary','culturalheritage',\
                      'dynasty','zodiacsign','thing','food','language',\
                     'material','chemicalelement','theorem','game',\
                     'creativework','historicalperiod','awardeventseries','plant',\
                     'product','fictionalthing','familyname','movie','symbol','curriculum']}

label2type = {}
for label,types in type2label.items():
    for j in types:
        label2type[j] = label
print(label2type)

In [7]:
print(kb_data.types)

{'internationalorganization', 'theorem', 'entertainmentperson', 'communicationmedium', 'symbol', 'curriculum', 'realestate', 'product', 'fictionalhuman', 'food', 'awardeventseries', 'language', 'place', 'animal', 'familyname', 'chemicalelement', 'academicdiscipline', 'fictionalthing', 'tool', 'medicaldepartmenttype', 'organism', 'thing', 'plant', 'country', 'historicalperiod', 'formula', 'nation', 'scientificorganization', 'culturalheritage', 'building', 'athlete', 'organization', 'tvshow', 'brand', 'movie', 'educationmajor', 'zodiacsign', 'collegeoruniversity', 'tvplay', 'material', 'creativework', 'astronomicalobject', 'dynasty', 'historicalperson', 'event', 'currency', 'medicalcondition', 'human', 'game', 'person', 'vocabulary'}


In [8]:
print(kb_data.predicate)

{'界面语言', '背景美術', '版本大小', '制作组', '学位学历', '武器特色', '母带师', '连载情况', '优酷上映', '车型尺寸', '植物类别', '形容的事物', '胞兄', '化名', '地铁日均客流', '著名太后', '本之义', '大致内容', '工作院系', '休闲设施', '经典语录', '著名高校', '功效主治', 'appstore', '预购时间', '著称于世', '特定动画人物', '出品机构', '面额', '作者朝代', '粤语读音', '关于媒商', '公交地铁', '组织代号', '矫正方法', '武术顾问', '采用网络', '其中之一', '最近更新内容', '英雄单位', '影片性质', '受众人群', '现节目主持', '录音后期', '代省长', '行政区划级别', '兴起国家', '燃烧三要素', '学校语文书', '中央界', '团体中角色', '装备特性', '青岛校区', '部活', '品牌主张', '达到', '特约编辑', '成立人', '中文直译', '现任总队长', '城市口号', '器乐录音师', '美国出版社', '拥有球', '最难忘的人', '学校吉祥物', '航速', '取景地点', '等级分类', '标识', '觉醒等级', '限制范围', '上映', '游戏容量', '功能特点', '线路类型', '动画后期', '日语版', '含义vi.', '烧制温度', '根椐地', '运营环境', '工作性质', '碎纸能力', '从警经历', '平均运动', '剧情年代', '注册资产', '区花区树', '五个首脑', 'imdb平分', '三十五回', '适用对向', '游戏言语', '经典语句', '召唤时间', '常用声线', '教义', '纲英文名', '魂兽形态', '最爱的点心', '男性阿斯泰坦', '主要运用', '策划/剧情', '统一政权', '曲目顺序', '正式登场', '珍品', '市民广场', '发卡银行', '译制', '颁奖礼司仪', '文学风格', '人物善恶', '感光器件', '导演人', '承制', '公司经营', '剧幕', '必经之地', '曾用假名', '支持帐号', '雀形目鸟', '全美累计票房', '北周', '提升途径

In [9]:
multi_ids = []
for kbs,ids in kb.kb2id.items():
    if len(ids)>1:
        multi_ids.append(kbs)


In [10]:
print(len(multi_ids))
print(multi_ids[:10])

for kbs in multi_ids[:1]:
#     tmp = []
    for ids in kb_data.kb2id[kbs]:
        print(kb_data.id2kb[ids])
#         tmp.append(kb.id2kb[ids])
#     print(tmp)
    

89694
['胜利', '张三的歌', '七里香', '王平', '王平将军', '树大招风', '王超', '无尽武道', '滕王阁序', '莲宗寺']
{'alias': ['胜利'], 'data': {'摘要': '英雄联盟胜利系列皮肤是拳头公司制作的具有纪念意义限定系列皮肤之一。拳头公司制作的具有纪念意义限定系列皮肤还包括英雄联盟冠军系列皮肤、msi季中冠军赛征服者系列以及英雄联盟全球总决赛冠军系列皮肤。每到赛季结束时，拳头公司都会制作胜利系列皮肤作为赛季奖励来认可那些在排位赛中勇猛拼搏达到黄金段位的玩家。', '制作方': 'riot games', '外文名': 'victorious', '来源': '英雄联盟', '中文名': '胜利', '属性': '虚拟', '义项描述': '游戏《英雄联盟》胜利系列限定皮肤'}, 'type': ['thing']}
{'alias': ['胜利'], 'data': {'摘要': '胜利，汉语词汇。拼音：shèng lì胜利，指达到预期的目的。与“失败”相对。有“成功”的意思，古代打仗成功称胜利，比赛夺冠胜利称“成功”。其他寓意也很广泛(如：一件事坚持到了最后也称胜利)。胜利在英语中都为victory [victory ]', '外文名': 'win', '反义词': '失败', '拼音': 'shèng lì', '中文名': '胜利', '释义': '获得成功或达到目的', '义项描述': '汉语词语', '标签': '文化'}, 'type': ['vocabulary']}
{'alias': ['胜利'], 'data': {'摘要': '《胜利》是由[英] 约瑟夫·康拉德所著一部讽喻小说，新华出版社出版发行。', '作者': '[英] 约瑟夫·康拉德', 'isbn': '9787516620762', '书名': '胜利', '出版社': '新华出版社', '义项描述': '[英] 约瑟夫·康拉德所著小说'}, 'type': ['thing']}
{'alias': ['胜利'], 'data': {'摘要': '《胜利》是动漫原声的音乐作品,收录在《火影忍者疾风传ost》专辑中。', '发行时间': '2007-03', '音乐时长': '1分47秒', '所属专辑': '火影忍者疾风传

In [22]:
from topsim import TopSim

In [23]:
ts = TopSim(kb.kb)

In [36]:
print(kb_data.kb[258037])

地铁南京


In [37]:
print(ts.search('南京',k=50))

[(1.0, [216188]), (0.6, [159073]), (0.5, [159076]), (0.4, [262414, 81948, 140024, 164598]), (0.3333333333333333, [36210, 168069, 258037, 286950, 296174, 27034, 81947, 81949, 95281, 113955, 123409, 123556, 125630, 130826, 133979, 140029, 157474, 164600, 171633, 172232, 185119, 186896, 214670, 221729, 224545, 227151, 228484, 236183, 239889, 244184, 249849, 258038, 266560, 266667, 275771, 279219, 282965, 286949, 298008]), (0.2857142857142857, [670, 38997, 241398, 29364])]


In [12]:
data = []
with open('./ccks2019_el/train.json') as f:
    for l in tqdm(f):
        _ = json.loads(l)
        tmp = {
            'text_id':_['text_id'],
            'text': _['text'].lower(),
            'mention_data': [(x['mention'].lower(), int(x['offset']), x['kb_id'])
                for x in _['mention_data'] if x['kb_id'] != 'NIL'
            ]
        }
        if tmp['mention_data']:
            data.append(tmp)

90000it [00:01, 48306.78it/s]


In [13]:
print(len(data))

85965


mode = 0
random_order = np.arange(len(data))
np.random.shuffle(random_order)train_data,dev_data,test_data
test_data = [data[j] for i, j in enumerate(random_order) if i % 101 == mode]
dev_data = [data[j] for i, j in enumerate(random_order) if i % 100 == mode and i % 101 != mode]
train_data = [data[j] for i, j in enumerate(random_order) if i % 100 != mode and i % 101 != mode]

print(len(train_data))
print(len(test_data))
print(len(dev_data))
result = len(train_data)+len(test_data)+len(dev_data)
print(result)
assert(result==len(data))

json.dump([train_data,dev_data,test_data],open('./data/all_data.json','w'))

In [ ]:
print(train_data[0])

In [18]:
import jieba

In [21]:
print(' '.join(jieba.cut('南京南站')))

南京 南站


In [72]:
cut_result = jieba.cut(train_data[0]['text'])
# print(' '.join(cut_result))
result = list(cut_result)
print(result)
# offset = [len((''.join(result[:i]))) for i in range(len(result))]
offset = [0]
for i in range(len(result)-1):
    offset.append(offset[-1]+len(result[i]))
for i in range(1,len(result)):
    result.append(''.join(result[i-3+1:i+1]))
    offset.append(offset[i-1])
#     offset.append(len((''.join(result[:i-3+1]))))
print(result)
print(offset)

['电影', '《', '淘金', '岁月', '》', '下载', ' ', '急求']
['电影', '《', '淘金', '岁月', '》', '下载', ' ', '急求', '', '电影《淘金', '《淘金岁月', '淘金岁月》', '岁月》下载', '》下载 ', '下载 急求']
[0, 2, 3, 5, 7, 8, 10, 11, 0, 2, 3, 5, 7, 8, 10]


In [76]:
ts_result = []
for i in result:
#     print(i)
    tmp = ts.search(i)
    if tmp and tmp[0][0] > 0.5:
        ts_result.append(tmp)
    else:
        ts_result.append([])
print(ts_result)

[[(1.0, [31107])], [], [], [(1.0, [7525])], [], [(1.0, [249645])], [], [], [], [], [(0.5714285714285714, [50472])], [(0.5714285714285714, [50472])], [], [], []]


In [196]:
kb_data = KB('./ccks2019_el/kb_data')

2295it [00:00, 22871.01it/s]

start loading kb_data...
construct id2kb dict...


399252it [00:21, 18435.18it/s]


construct kb2id dict...
KB DATA INFORMATION
TOKEN SIZE:303375
ID SIZE:399233
TYPE SIZE:51
PREDICATE SIZE:41841


In [217]:
class ngram_search(object):
    def __init__(self,data,kb,ngram = 2,similarity = 0.5):
        self.n = ngram
        self.similarity = similarity
        self.data = data
        self.kb = kb
        self.cut_data,self.offset = self.cut_words()
        self.ts = TopSim(self.kb)
        self.candidates = self.get_candidates(self.similarity)
        self.cand_name,self.cand_off = self.get_candidates_name()
    def cut_words(self):
        print('starting build ngram list')
        print('ngram',self.n)
        result = []
        offset = []
        for d in tqdm(self.data):
#             print(d)
#             print(' '.join(jieba.cut(d)))
            tmp = list(jieba.cut(d))
            n = len(tmp)
            tmp_off = [0]
#             tmp_off = [len(''.join(tmp[:i])) for i in range(len(tmp))]
            for i in range(len(tmp)-1):
                tmp_off.append(tmp_off[-1]+len(tmp[i]))
            for j in range(2,self.n+1):
                for i in range(j-1,n):
                    tmp.append(''.join(tmp[i-j+1:i+1]))
                    tmp_off.append(tmp_off[i-j+1])
#                     tmp_off.append(''.join(tmp[:i-n+1]))
            result.append(tmp)
            offset.append(tmp_off)
        return result,offset
    def get_candidates(self,similarity = 0.5):
        self.similarity = similarity
        print('starting build candidates list')
        print('similarity:',self.similarity)
        candidates = []
        for dt in tqdm(self.cut_data):
            ts_result = []
            for i in dt:
                tmp = ts.search(i)
                if tmp and tmp[0][0] > self.similarity:
                    ts_result.append(tmp)
                else:
                    ts_result.append([])
            candidates.append(ts_result)
        return candidates
    def get_candidates_name(self):
        print('starting get candidates name and offset')
        cand_name = []
        cand_offset = []
        for i in tqdm(range(len(self.candidates))):
            cand = []
            off = []
            for j in range(len(self.candidates[i])):
                if self.candidates[i][j]:
#                     print(self.candidates[i][j])
#                     print(self.candidates[i][j][0][1][0])
#                     print(self.kb[self.candidates[i][j][0][1][0]])
                    cand.append(self.kb[self.candidates[i][j][0][1][0]])
                    off.append(self.offset[i][j])
            cand_name.append(cand)
            cand_offset.append(off)
        return cand_name,cand_offset



In [218]:
dev_x = [i['text'] for i in dev_data]
print(type(dev_x[0]))

<class 'str'>


In [244]:
ns_test = ngram_search([dev_x[0]],kb_data.kb)
print(ns_test.candidates)
print(ns_test.cand_name)




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 925.08it/s]

starting build ngram list
ngram 2





  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 19.83it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 1892.74it/s]

starting build candidates list
starting get candidates name and offset
[[[], [(1.0, [44529])], [(1.0, [4657])], [], [], [(1.0, [10911])], [], [(1.0, [48458])], [], [], [(1.0, [35639])], [(1.0, [108397])], [], [], [], [], [(1.0, [44529])], [(1.0, [4657])], [], [(1.0, [4605])], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [(1.0, [4605])]]]
[['铁', '梨花', '结局', '人', '的', '失落', '铁', '梨花', '铁梨花', '铁梨花']]


In [243]:
print(dev[0])
print(en[0])
print(ns_test.cand_name)
print(ns_test.cand_off)

《铁梨花》,结局让人有着说不出的失落。 - 铁梨花
['铁梨花', '结局', '失落', '铁梨花']
[['铁', '梨花', '结局', '人', '的', '失落', '铁', '梨花', '铁梨花', '铁梨花']]
[[1, 2, 6, 9, 15, 16, 22, 23, 1, 22]]


In [240]:
a = kb_data.kb2id['铁']
print(a)
for i in a:
    print(kb_data.id2kb[i])

['44637', '76121', '157702', '209828', '213677', '216377', '300998', '307908', '349237', '354871', '370145', '373877', '375248']
{'alias': ['铁'], 'data': {'摘要': '铁，中药材名。本品为一种灰黑色的金属。主要由赤铁矿、褐铁矿、磁铁矿等炼出。由于含碳量的不同，可分为生铁(含碳量在1.7％以上)、熟铁(含碳量在0.2％以下)和钢铁(含碳量在0.2～1.7％之间)三种。功能主治为：镇心平肝，消痈解毒。治惊痫，癫狂，痈毒。1.《本经》：主坚肌耐痛。2.《别录》：主疗下部及脱肛。主金疮，烦满，热中，胸膈气塞，食不化。3.《本草拾遗》：主贼风，烧赤投酒中热服之。4.《纲目》：散瘀血，消丹毒。5.《本草汇言》：平肝气，安惊痫。6.《本草备要》：镇心平肝，定惊疗狂，消痈解毒。', '汉语拼音': 'tie', '中文名': '铁', '义项描述': '铁'}, 'type': ['thing']}
{'alias': ['铁'], 'data': {'摘要': '铁，中药材名。本品为一种灰黑色的金属。主要由赤铁矿、褐铁矿、磁铁矿等炼出。由于含碳量的不同，可分为生铁(含碳量在1.7％以上)、熟铁(含碳量在0.2％以下)和钢铁(含碳量在0.2～1.7％之间)三种。功能主治为：镇心平肝，消痈解毒。治惊痫，癫狂，痈毒。1.《本经》：主坚肌耐痛。2.《别录》：主疗下部及脱肛。主金疮，烦满，热中，胸膈气塞，食不化。3.《本草拾遗》：主贼风，烧赤投酒中热服之。4.《纲目》：散瘀血，消丹毒。5.《本草汇言》：平肝气，安惊痫。6.《本草备要》：镇心平肝，定惊疗狂，消痈解毒。', '汉语拼音': 'tie', '中文名': '铁', '义项描述': '铁'}, 'type': ['thing']}
{'alias': ['铁'], 'data': {'摘要': '从这幅图可以看出，sc(生存战争)的矿物分布不仅与层数有关，也与生态环境有关，也就是别想在土层一下找硝石，同时，并不是岩石层就没有泥土由图可得：煤炭的分部为由上到下从地表面向下逐渐减少。硝石(也可以是氮肥，因为既可以做火药又可以做肥料，但是原版英文为硝石)的分布为大量分布于沙子下方，沙石

In [220]:
for i in range(2,6):
    ns_test = ngram_search([dev_x[0]],kb_data.kb,i)
    print(ns_test.cut_data)
    print(ns_test.cand_name)

100%|██████████| 1/1 [00:00<00:00, 1589.96it/s]

starting build ngram list
ngram 2



  0%|          | 0/1 [00:00<?, ?it/s]

starting build candidates list
starting get candidates name and offset
[['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花', '《铁', '铁梨花', '梨花》', '》,', ',结局', '结局让', '让人', '人有着', '有着说不出', '说不出的', '的失落', '失落。', '。 ', ' -', '- ', ' 铁', '铁梨花']]
[['铁', '梨花', '结局', '人', '的', '失落', '铁', '梨花', '铁梨花', '铁梨花']]
starting build ngram list
ngram 3


100%|██████████| 1/1 [00:00<00:00, 1383.80it/s]

starting build candidates list
starting get candidates name and offset
[['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花', '《铁', '铁梨花', '梨花》', '》,', ',结局', '结局让', '让人', '人有着', '有着说不出', '说不出的', '的失落', '失落。', '。 ', ' -', '- ', ' 铁', '铁梨花', '《铁梨花', '铁梨花》', '梨花》,', '》,结局', ',结局让', '结局让人', '让人有着', '人有着说不出', '有着说不出的', '说不出的失落', '的失落。', '失落。 ', '。 -', ' - ', '- 铁', ' 铁梨花']]
[['铁', '梨花', '结局', '人', '的', '失落', '铁', '梨花', '铁梨花', '铁梨花']]
starting build ngram list
ngram 4



100%|██████████| 1/1 [00:00<00:00, 4604.07it/s]

starting build candidates list
starting get candidates name and offset



100%|██████████| 1/1 [00:00<00:00, 441.09it/s]

[['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花', '《铁', '铁梨花', '梨花》', '》,', ',结局', '结局让', '让人', '人有着', '有着说不出', '说不出的', '的失落', '失落。', '。 ', ' -', '- ', ' 铁', '铁梨花', '《铁梨花', '铁梨花》', '梨花》,', '》,结局', ',结局让', '结局让人', '让人有着', '人有着说不出', '有着说不出的', '说不出的失落', '的失落。', '失落。 ', '。 -', ' - ', '- 铁', ' 铁梨花', '《铁梨花》', '铁梨花》,', '梨花》,结局', '》,结局让', ',结局让人', '结局让人有着', '让人有着说不出', '人有着说不出的', '有着说不出的失落', '说不出的失落。', '的失落。 ', '失落。 -', '。 - ', ' - 铁', '- 铁梨花']]
[['铁', '梨花', '结局', '人', '的', '失落', '铁', '梨花', '铁梨花', '铁梨花']]
starting build ngram list
ngram 5
starting build candidates list
starting get candidates name and offset
[['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花', '《铁', '铁梨花', '梨花》', '》,', ',结局', '结局让', '让人', '人有着', '有着说不出', '说不出的', '的失落', '失落。', '。 ', ' -', '- ', ' 铁', '铁梨花', '《铁梨花', '铁梨花》', '梨花》,', '》,结局', ',结局让', '结局让人', '让人有着', '人有着说不出', '有着说不出的', '说不出的失落', '的失落。', '失落。 ', '。 -', ' - ', '- 铁', ' 铁梨花',

In [195]:
ns = ngram_search(dev_x,kb_data.kb)
candidates = ns.candidates
print(candidates[0])
offset = ns.offset
print(offset[0])
print(ns.cut_data[0])
cand = []
off = []
for i in range(len(candidates[0])):
    if candidates[0][i]:
#         tmp = 
        cand.append(kb_data.kb[candidates[0][i][0][1][0]])
        off.append(offset[0][i])
print(cand)
print(off)

100%|██████████| 851/851 [00:00<00:00, 4589.01it/s]

starting build ngram list
ngram 2



  0%|          | 3/851 [00:00<00:30, 28.22it/s]

starting build candidates list


  5%|▍         | 42/851 [00:00<00:02, 351.25it/s]

...


可夫
[(1.0, [15906])]
15906
孔子
[(1.0, [101527])]
101527
76
[(1.0, [45123])]
45123
代
[(1.0, [126343])]
126343
孙女
[(1.0, [292546])]
292546
王叔铭
[(1.0, [249062])]
249062
王光美
[(1.0, [35639])]
35639
的
[(1.0, [144750])]
144750
“
[(1.0, [97288])]
97288
父女
[(1.0, [144752])]
144752
”
[(1.0, [62943])]
62943
_
[(1.0, [40897])]
40897
恐惧
[(1.0, [26174])]
26174
日
[(1.0, [36350])]
36350
王莽
[(1.0, [37411])]
37411
...
[(1.0, [227167])]
227167
公安
[(1.0, [93913])]
93913
传播
[(1.0, [115145])]
115145
彭耀春
[(1.0, [279462])]
279462
摘要
[(1.0, [20046])]
20046
书评
[(1.0, [165814])]
165814
试读
[(1.0, [46956])]
46956
金灿灿
[(1.0, [63548])]
63548
广场
[(1.0, [5495])]
5495
舞
[(1.0, [14045])]
14045
美丽
[(1.0, [35639])]
35639
的
[(1.0, [64190])]
64190
草原
[(1.0, [14045])]
14045
美丽
[(1.0, [35639])]
35639
的
[(1.0, [47072])]
47072
姑娘
[(1.0, [63764])]
63764
附
[(1.0, [50216])]
50216
背面
[(1.0, [94075])]
94075
演
[(1.0, [64941])]
64941
广场舞
[(1.0, [2138])]
2138
李建平
[(1.0, [57782])]
57782
在
[(1.0, [222245])]
222245
深圳
[(1.0, [62943])]
6294

 93%|█████████▎| 790/851 [00:02<00:00, 308.52it/s]

大
[(1.0, [142553])]
142553
2017年
[(1.0, [10071])]
10071
·
[(1.0, [287903])]
287903
图片
[(1.0, [145729])]
145729
价格
[(1.0, [161639])]
161639
品牌
[(1.0, [253145])]
253145
报价
[(1.0, [37031])]
37031
南渡北归
[(1.0, [122660])]
122660
房
[(1.0, [62943])]
62943
_
[(1.0, [186224])]
186224
珠海
[(1.0, [86617])]
86617
房产
[(1.0, [190175])]
190175
信息网
[(1.0, [10071])]
10071
·
[(1.0, [104119])]
104119
梧桐
[(1.0, [14019])]
14019
路
[(1.0, [186224])]
186224
珠海
[(1.0, [293565])]
293565
百姓网
[(0.625, [16126])]
16126
房地产信息网
[(0.6, [165626])]
165626
省委常委
[(1.0, [81050])]
81050
组长
[(1.0, [177158])]
177158
准则
[(1.0, [150114])]
150114
条例
[(1.0, [141779])]
141779
专题
[(1.0, [123067])]
123067
景观
[(1.0, [148077])]
148077
设计
[(1.0, [123067])]
123067
景观
[(1.0, [150936])]
150936
张亚萍
[(1.0, [123277])]
123277
梅洛
[(1.0, [279462])]
279462
摘要
[(1.0, [37411])]
37411
...
[(1.0, [200])]
200
母亲
[(1.0, [35639])]
35639
的
[(1.0, [31107])]
31107
电影
[(1.0, [167338])]
167338
高清
[(1.0, [189722])]
189722
在线
[(1.0, [105483])]
105483
观看
[(1.0, 

100%|██████████| 851/851 [00:02<00:00, 292.21it/s]


[(1.0, [63710])]
63710
学
[(1.0, [140109])]
140109
系列
[(1.0, [216769])]
216769
袁其刚
[(1.0, [279462])]
279462
摘要
[(1.0, [32444])]
32444
帮帮我
[(1.0, [32442])]
32442
爱神
[(1.0, [262813])]
262813
剧照
[(1.0, [1038])]
1038
曝光
[(1.0, [32441])]
32441
帮帮我爱神
[(1.0, [8052])]
8052
王恺
[(1.0, [30596])]
30596
—
[(1.0, [30596])]
30596
—
[(1.0, [30596])]
30596
—
[(1.0, [30596])]
30596
—
[(1.0, [69952])]
69952
武汉大学
[(1.0, [261353])]
261353
经济
[(1.0, [150224])]
150224
管理
[(1.0, [12370])]
12370
学院
[(1.0, [28542])]
28542
——
[(0.5555555555555556, [107817])]
107817
管理科学与工程
[(1.0, [28542])]
28542
——
[(0.5714285714285714, [69952])]
69952
武汉大学
[(0.6666666666666666, [78248])]
78248
武汉大学经济学
[(1.0, [44146])]
44146
吧
[(1.0, [159854])]
159854
肿瘤
[(0.6666666666666666, [37635])]
37635
君君
[(1.0, [94163])]
94163
里
[(1.0, [9716])]
9716
有
[(1.0, [55608])]
55608
一个
[(1.0, [144750])]
144750
“
[(1.0, [141616])]
141616
攻受
[(1.0, [144752])]
144752
”
[(1.0, [12031])]
12031
演员
[(1.0, [37411])]
37411
...
[(1.0, [32468])]
32468
知识
[(1

In [172]:
dev_x[0]

'《铁梨花》,结局让人有着说不出的失落。 - 铁梨花'

In [180]:
print(candidates[0])

[[], [(1.0, [44529])], [(1.0, [4657])], [], [], [(1.0, [10911])], [], [(1.0, [48458])], [], [], [(1.0, [35639])], [(1.0, [108397])], [], [], [], [], [(1.0, [44529])], [(1.0, [4657])], [], [(1.0, [4605])], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [(1.0, [4605])]]


In [ ]:
cand_name = ns.cand_name

In [151]:
print(dev_data[0])

{'text_id': '59668', 'text': '《铁梨花》,结局让人有着说不出的失落。 - 铁梨花', 'mention_data': [('铁梨花', 1, '143416'), ('结局', 6, '335878'), ('失落', 16, '286921'), ('铁梨花', 22, '143416')]}


In [167]:
en = []
for i in dev_data:
    tmp = []
    for j in i['mention_data']:
        tmp.append(j[0])
    en.append(tmp)

In [168]:
print(en[0])

['铁梨花', '结局', '失落', '铁梨花']


recall = []
for i in range(len(en)):
    tp = 0
    fp = 0
    for j in en[i]:
        if j in cand_name[i]:
            tp += 1
        else:
            fp += 1
    recall.append(tp/(tp+fp))
print(sum(recall)/len(recall))

In [228]:
recall_all = []
for i in range(2,10):
    print('ngram:',i)
    ns = ngram_search(dev_x,kb_data.kb,i)
    recall = []
    cand_name = ns.cand_name
#     print(len)
    for i in tqdm(range(len(en))):
        tp = 0
        fp = 0
        for j in en[i]:
            if j in cand_name[i]:
                tp += 1
            else:
                fp += 1
        recall.append(tp/(tp+fp))
    print(sum(recall)/len(recall))
    recall_all.append(recall)

 44%|████▍     | 377/851 [00:00<00:00, 3763.14it/s]

ngram: 2
starting build ngram list
ngram 2


  0%|          | 3/851 [00:00<00:31, 27.22it/s]

starting build candidates list


  0%|          | 0/851 [00:00<?, ?it/s]

...

starting build candidates list


100%|██████████| 851/851 [00:00<00:00, 29826.38it/s]


starting get candidates name and offset


100%|██████████| 851/851 [00:00<00:00, 175631.19it/s]

0.9387937627068058


In [235]:
recall_all = []
ratio_all = []
for i in range(0,5):
#     print('ngram:',i)
    print('similarity',i/10)
    ns = ngram_search(dev_x,kb_data.kb,4,i/10)
    recall = []
    cand_name = ns.cand_name
    ratio = []
#     print(len)
    for i in tqdm(range(len(en))):
        tp = 0
        fp = 0
        for j in en[i]:
            if j in cand_name[i]:
                tp += 1
            else:
                fp += 1
        recall.append(tp/(tp+fp))
        ratio.append(len(cand_name[i])/len(en[i]))
    print(sum(recall)/len(recall))
    print(sum(ratio)/len(ratio))
    recall_all.append(recall)
    ratio_all.append(ratio)




  0%|          | 0/851 [00:00<?, ?it/s]


 46%|████▌     | 388/851 [00:00<00:00, 3868.92it/s]

similarity 0.0
starting build ngram list
ngram 4





 89%|████████▊ | 755/851 [00:00<00:00, 3802.02it/s]


100%|██████████| 851/851 [00:00<00:00, 3616.15it/s]


  0%|          | 0/851 [00:00<?, ?it/s]


  0%|          | 2/851 [00:00<00:54, 15.46it/s]

starting build candidates list





  0%|          | 4/851 [00:00<00:56, 14.99it/s]


  1%|          | 6/851 [00:00<00:55, 15.31it/s]


  1%|          | 9/851 [00:00<00:47, 17.91it/s]


  2%|▏         | 13/851 [00:00<00:41, 20.29it/s]


  2%|▏         | 15/851 [00:00<00:54, 15.38it/s]


  2%|▏         | 17/851 [00:00<00:52, 15.78it/s]


  2%|▏         | 19/851 [00:01<01:05, 12.71it/s]

 93%|█████████▎| 795/851 [01:02<00:03, 14.74it/s]


  2%|▏         | 21/851 [00:01<01:09, 11.97it/s]


  3%|▎         | 23/851 [00:01<01:03, 12.97it/s]


  3%|▎         | 27/851 [00:01<00:57, 14.30it/s]


  4%|▎         | 30/851 [00:01<00:52, 15.68it/s]


  4%|▍         | 32/851 [00:01<00:49, 16.61it/s]


  4%|▍         | 34/851 [00:02<00:50, 16.15it/s]


  4%|▍         | 37/851 [00:02<00:47, 17.12it/s]


  5%|▍         | 41/851 [00:02<00:44, 18.32it/s]


  5%|▌         | 44/851 [00:02<00:43, 18.51it/s]


  5%|▌         | 46/851 [00:02<00:45, 17.60it/s]


  6%|▌         | 48/851 [00:02<00:47, 16.82it/s]


  6%|▌         | 50/851 [00:03<

...




  0%|          | 4/851 [00:00<00:50, 16.68it/s]


  1%|          | 6/851 [00:00<00:49, 17.02it/s]


  1%|          | 10/851 [00:00<00:40, 20.53it/s]


  2%|▏         | 13/851 [00:00<00:37, 22.32it/s]


  2%|▏         | 16/851 [00:00<00:43, 19.24it/s]


  2%|▏         | 18/851 [00:00<00:58, 14.32it/s]


  2%|▏         | 20/851 [00:01<00:58, 14.19it/s]


  3%|▎         | 22/851 [00:01<01:02, 13.29it/s]


  3%|▎         | 26/851 [00:01<00:51, 16.06it/s]


  3%|▎         | 28/851 [00:01<00:56, 14.52it/s]


  4%|▎         | 30/851 [00:01<00:54, 15.07it/s]


  4%|▍         | 33/851 [00:01<00:51, 15.85it/s]


  4%|▍         | 36/851 [00:02<00:46, 17.46it/s]


  5%|▍         | 40/851 [00:02<00:38, 20.82it/s]


  5%|▌         | 43/851 [00:02<00:40, 19.80it/s]


  5%|▌         | 46/851 [00:02<00:44, 17.95it/s]


  6%|▌         | 49/851 [00:02<00:49, 16.21it/s]


  6%|▌         | 51/851 [00:02<00:47, 16.72it/s]


  6%|▌         | 53/851 [00:02<00:47, 16.68it/s]


  6%|▋         | 55/851 [00:03

 47%|████▋     | 401/851 [00:22<00:25, 17.74it/s]


 47%|████▋     | 404/851 [00:22<00:23, 19.04it/s]


 48%|████▊     | 407/851 [00:22<00:21, 20.23it/s]


 48%|████▊     | 410/851 [00:22<00:37, 11.87it/s]


 49%|████▊     | 413/851 [00:23<00:35, 12.50it/s]


 49%|████▉     | 417/851 [00:23<00:29, 14.90it/s]


 50%|████▉     | 422/851 [00:23<00:22, 18.67it/s]


 50%|████▉     | 425/851 [00:23<00:25, 16.79it/s]


 50%|█████     | 428/851 [00:23<00:25, 16.47it/s]


 51%|█████     | 431/851 [00:23<00:23, 18.12it/s]


 51%|█████     | 434/851 [00:24<00:24, 17.06it/s]


 51%|█████▏    | 437/851 [00:24<00:22, 18.54it/s]


 52%|█████▏    | 440/851 [00:24<00:19, 20.81it/s]


 52%|█████▏    | 444/851 [00:24<00:17, 23.93it/s]


 53%|█████▎    | 448/851 [00:24<00:15, 25.75it/s]


 53%|█████▎    | 451/851 [00:24<00:15, 25.78it/s]


 53%|█████▎    | 454/851 [00:24<00:14, 26.69it/s]


 54%|█████▎    | 457/851 [00:24<00:16, 23.95it/s]


 54%|█████▍    | 460/851 [00:25<00:16, 23.01it/s]


 54%|█████▍ 

 94%|█████████▍| 804/851 [00:44<00:02, 20.67it/s]


 95%|█████████▍| 807/851 [00:44<00:02, 21.03it/s]


 95%|█████████▌| 810/851 [00:45<00:01, 21.09it/s]


 96%|█████████▌| 813/851 [00:45<00:01, 21.61it/s]


 96%|█████████▌| 816/851 [00:45<00:01, 22.75it/s]


 96%|█████████▌| 819/851 [00:45<00:01, 23.86it/s]


 97%|█████████▋| 822/851 [00:45<00:01, 21.68it/s]


 97%|█████████▋| 825/851 [00:45<00:01, 20.22it/s]


 97%|█████████▋| 828/851 [00:45<00:01, 20.94it/s]


 98%|█████████▊| 831/851 [00:45<00:00, 22.65it/s]


 98%|█████████▊| 834/851 [00:46<00:00, 22.27it/s]


 98%|█████████▊| 837/851 [00:46<00:00, 21.99it/s]


 99%|█████████▊| 840/851 [00:46<00:00, 22.64it/s]


 99%|█████████▉| 843/851 [00:46<00:00, 23.08it/s]


 99%|█████████▉| 846/851 [00:46<00:00, 18.89it/s]


100%|█████████▉| 849/851 [00:46<00:00, 18.97it/s]


100%|██████████| 851/851 [00:46<00:00, 18.53it/s]


  0%|          | 0/851 [00:00<?, ?it/s]


100%|██████████| 851/851 [00:00<00:00, 37125.84it/s]




starting get candidates name and offset


  0%|          | 0/851 [00:00<?, ?it/s]


100%|██████████| 851/851 [00:00<00:00, 196053.65it/s]

In [237]:
for i in range(len(ratio_all)):
    print(sum(ratio_all[i])/len(ratio_all[i]))
    print(sum(recall_all[i])/len(recall_all[i]))

18.855327532501448
0.9521725141290355
18.590112939025982
0.9521725141290355
15.693984201592894
0.951447876447876
10.786663682315858
0.9491797698319432
6.31460793091228
0.9422406878928613


In [223]:
cut_data = ns.cut_data

for i in range(len(recall)):
    if recall[i] < 1:
#         print(cand_name[i])
#         print(dev_data[i])
        for j in en[i]:
            if j not in cand_name[i]:
                if j in cut_data[i]:
                    print(cand_name[i])
                    print(cut_data[i])
                    print(dev_data[i])
        
        

['绿洲', '影评', 'mtime', '时光', '网', 'mtime', 'mtime', '时光网', 'mtime']
['绿洲', ' ', '影评', '╟', ' ', 'mtime', '时光', '网', '绿洲 ', ' 影评', '影评╟', '╟ ', ' mtime', 'mtime时光', '时光网', '绿洲 影评', ' 影评╟', '影评╟ ', '╟ mtime', ' mtime时光', 'mtime时光网', '绿洲 影评╟', ' 影评╟ ', '影评╟ mtime', '╟ mtime时光', ' mtime时光网', '绿洲 影评╟ ', ' 影评╟ mtime', '影评╟ mtime时光', '╟ mtime时光网']
{'text_id': '9107', 'text': '绿洲 影评╟ mtime时光网', 'mention_data': [('绿洲', 0, '159221'), ('影评', 3, '22265'), ('mtime时光网', 7, '153474')]}
['守护者', '世纪', '精彩', '片段', '俄罗斯', '动作', '科幻片', '世纪战元', '世纪战元', '世纪战元', '守护者：世纪战元']
['《', '守护者', ':', '世纪', '战元', '》', '精彩', '片段', ' ', '俄罗斯', '动作', '科幻片', '《守护者', '守护者:', ':世纪', '世纪战元', '战元》', '》精彩', '精彩片段', '片段 ', ' 俄罗斯', '俄罗斯动作', '动作科幻片', '《守护者:', '守护者:世纪', ':世纪战元', '世纪战元》', '战元》精彩', '》精彩片段', '精彩片段 ', '片段 俄罗斯', ' 俄罗斯动作', '俄罗斯动作科幻片', '《守护者:世纪', '守护者:世纪战元', ':世纪战元》', '世纪战元》精彩', '战元》精彩片段', '》精彩片段 ', '精彩片段 俄罗斯', '片段 俄罗斯动作', ' 俄罗斯动作科幻片', '《守护者:世纪战元', '守护者:世纪战元》', ':世纪战元》精彩', '世纪战元》精彩片段', '战元》精彩片段 ', '》精彩片段 俄罗斯', '精彩片段 俄罗斯动作

In [224]:
ts = TopSim(kb_data.kb)

In [226]:
print(ts.search('神薙'))

[(1.0, [61124])]


In [158]:
kb.kb2id['的']

['36797', '60209', '70607', '343599']

In [159]:
for i in ['36797', '60209', '70607', '343599']:
    print(kb.id2kb[i])
    cand_name = ns.cand_name

{'alias': ['的', '马廷义'], 'data': {'摘要': '马廷义，男，1956年1月出生，汉族，河南明泰铝业股份有限公司董事长兼总经理，巩义市第三届人大常务委员会委员。1997年，他组建了河南明泰铝业有限公司。10年来，他依靠自有资金，实行滚动发展，使当年名不见经传的小厂成为拥有注册资金3.15亿元、总资产达到13亿元、占地面积24.6万平方米、职工2000余人的巩义30强企业，累计向国家上交税收1亿多元，安置下岗职工与社会劳动力1800余人。', '国籍': '中国', '出生日期': '1956年1月', '民族': '汉族', '别名': '的', '中文名': '马廷义', '义项描述': '马廷义', '标签': '行业人物'}, 'type': ['human']}
{'alias': ['喷喉爽', '的'], 'data': {'地点': '云南', '民族': '彝族', '中文名': '的', '属性': '药物', '义项描述': '喷喉爽', '标签': '食品'}, 'type': ['human']}
{'alias': ['的'], 'data': {'摘要': '的是一个汉语汉字，普通话读音是de,dí,dì,dī(粤语读音为dik1)，其居现代汉语使用量之首，有着漫长的演变史。用在定语的后面，也可以用在主谓短语中间，现代汉语56008个常用词 “的”字用最多的一个字。', '外文名': 'of', '词性': '名词、助词、代词', '笔顺编号': '32511354', '五笔86,98': 'r(一级简码)、rqyy(全码)', '普通话拼音': 'de,dí,dì,dī', '总笔画数': '8', '粤语拼音': 'dik1', '部首': '白', '仓颉': 'hapi', '中文名': '的', '郑码': 'nkrs', '二笔': 'd w t v(全码)、d(一级简码)', '造字法': '形声。本作“旳”，从日，勺声', 'unicode': 'cjk 统一汉字 u+7684', '结构': '左右结构', '四角号码': '27620', '义项描述': '汉语汉字', '标签': '语言'}, 'type': ['vocabulary']}
{'alias': ['的'], 'data

In [ ]:
def build_vocabs(datasets):
    token_vocabs = set()
    for 

In [ ]:
def build_char2id(token_vocabs):
    char2id = {}
    for 

In [ ]:
import numpy as np
import os

In [ ]:
UNK = "$UNK$"
NUM = "$NUM$"
NONE = "O"

In [ ]:
# special error message
class MyIOError(Exception):
    def __init__(self, filename):
        # custom error message
        message = """
ERROR: Unable to locate file {}.

FIX: Have you tried running python build_data.py first?
This will build vocab file from your train, test and dev sets and
trimm your word vectors.
""".format(filename)
        super(MyIOError, self).__init__(message)

In [ ]:
class Dataset(object):
    '''
    读文件，生成matrix
    '''
    def __init__(self,filename):
        
    def __len__(self,)
        
    